In [1]:
import scapy

In [ ]:
# Specify the PCAP trace file

pcap_trace = "iot_23_datasets_full/opt/Malware-Project/BigDataset/IoTScenarios/CTU-Honeypot-Capture-4-1/2018-09-14-13-40-25-Philips-Hue-Bridge.pcap"



In [3]:
# Using Scapy to read a PCAP file

from scapy.utils import PcapReader
from scapy.layers.inet import IP, TCP, UDP

# Metrics to collect about a specific PCAP trace
packet_count = 0
total_bytes = 0
protocol_counts = {"TCP": 0, "UDP": 0, "ICMP": 0, "OTHER": 0}
unique_src_ips = set()
unique_dst_ips = set()

with PcapReader(pcap_trace) as pcap:
    for pkt in pcap:
        packet_count += 1
        total_bytes += len(pkt)

        # IP-level info
        if IP in pkt:
            ip = pkt[IP]
            unique_src_ips.add(ip.src)
            unique_dst_ips.add(ip.dst)

            # Protocol counting
            if TCP in pkt:
                protocol_counts["TCP"] += 1
            elif UDP in pkt:
                protocol_counts["UDP"] += 1
            elif ip.proto == 1:
                protocol_counts["ICMP"] += 1
            else:
                protocol_counts["OTHER"] += 1

In [4]:
# Total metrics about packets

print("Total packets:", packet_count)
print("Total bytes:", total_bytes)
print("Average packet size:", total_bytes / packet_count)

print("\nProtocol counts:", protocol_counts)
print("Unique source IPs:", len(unique_src_ips))
print("Unique destination IPs:", len(unique_dst_ips))

Total packets: 8573
Total bytes: 2292136
Average packet size: 267.3668494109413

Protocol counts: {'TCP': 4999, 'UDP': 1278, 'ICMP': 325, 'OTHER': 0}
Unique source IPs: 64
Unique destination IPs: 73


In [5]:
#From the following website: https://medium.com/@ishankarunanayake/extracting-features-from-a-pcap-file-and-writing-to-csv-txt-using-python-c7630ac6322a

from scapy.utils import RawPcapReader
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
pcap_file = pcap_trace
output_file = 'example_pcap_putput.txt'
def process_pcap(in_file, out_file):
    f = open(out_file, "w+")
    count = 0
    first_timestamp = 0
    line_list = []
    
    # Looping through all the packets in the PCAP
    for (pkt_data, pkt_metadata,) in RawPcapReader(in_file):
        try:
            ether_pkt = Ether(pkt_data)
        except:
            continue

        if IP not in ether_pkt:
            continue
        ip_pkt = ether_pkt[IP]
        src = ip_pkt.src # Get the source IP
        dst = ip_pkt.dst # Get the destination IP
        
        # Calculate the relative timestamp of packets compared to the first packet
        timestamp = pkt_metadata.sec + (pkt_metadata.usec)/1000000
        if count == 0:
            first_timestamp = timestamp
            relative_timestamp = 0.0
        else:
            relative_timestamp = timestamp - first_timestamp
        
        pkt_size = pkt_metadata.caplen # Get packet size
        count += 1

        sport,dport = -1,-1
        if ip_pkt.haslayer(TCP):
            sport = ip_pkt[TCP].sport
            dport = ip_pkt[TCP].dport
        elif ip_pkt.haslayer(UDP):
            sport = ip_pkt[UDP].sport
            dport = ip_pkt[UDP].dport
        line = src + "," + dst + "," + str(round(relative_timestamp, 6)) + "," + str(pkt_size) + "," + str(ip_pkt.proto) + "," + str(sport) + "," + str(dport) + "\n"
        line_list.append(line)
    
    f.writelines(line_list)
    f.close()
    
process_pcap(pcap_file, output_file)